In [1]:
#! pip install sqlalchemy
#! pip install PyMySQL

In [2]:
import pymysql
from sqlalchemy import create_engine, text
import pandas as pd
import getpass
password = getpass.getpass()

········


In [3]:
connection_string = f'mysql+pymysql://root:' + password + '@127.0.0.1:3306/bank'
engine = create_engine(connection_string)

In [4]:
try:
    engine = create_engine(connection_string, echo=True)
    with engine.connect() as connection:
        print("✅ Successfully connected to MySQL!")
except Exception as e:
    print("❌ Connection failed:", e)

2025-03-19 14:15:34,338 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2025-03-19 14:15:34,339 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-19 14:15:34,340 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2025-03-19 14:15:34,340 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-19 14:15:34,341 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2025-03-19 14:15:34,342 INFO sqlalchemy.engine.Engine [raw sql] {}
✅ Successfully connected to MySQL!


In [5]:
skytrax_db = engine.execute("CREATE DATABASE IF NOT EXISTS SkytraxReviews")

2025-03-19 14:15:34,350 INFO sqlalchemy.engine.Engine CREATE DATABASE IF NOT EXISTS SkytraxReviews
2025-03-19 14:15:34,351 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-19 14:15:34,354 INFO sqlalchemy.engine.Engine COMMIT


In [6]:
with engine.connect() as conn:
    conn.execute(text("USE SkytraxReviews;"))  
    result = conn.execute(text("SHOW TABLES;"))
    print(result.fetchall())

2025-03-19 14:15:34,359 INFO sqlalchemy.engine.Engine USE SkytraxReviews;
2025-03-19 14:15:34,360 INFO sqlalchemy.engine.Engine [generated in 0.00091s] {}
2025-03-19 14:15:34,362 INFO sqlalchemy.engine.Engine SHOW TABLES;
2025-03-19 14:15:34,362 INFO sqlalchemy.engine.Engine [generated in 0.00049s] {}
[]


In [7]:
query = '''
        CREATE TABLE IF NOT EXISTS reviews (
            id INT AUTO_INCREMENT PRIMARY KEY,
            date_published DATE,
            aircraft VARCHAR(255),
            type_of_traveller VARCHAR(100),
            seat_type VARCHAR(50),
            route VARCHAR(255),
            date_flown DATE NULL,
            seat_comfort_ranking INT NULL,
            cabin_staff_service_ranking INT NULL,
            food_beverages_ranking INT NULL,
            ground_services_ranking INT NULL,
            value_for_money INT,
            recommended VARCHAR(10),
            verification_status VARCHAR(50),
            review_text_clean TEXT,
            flight_type VARCHAR(50),
            departure VARCHAR(100),
            destination VARCHAR(100),
            layover VARCHAR(100) NULL,
            sentiment_score FLOAT,
            sentiment VARCHAR(10),
            keywords TEXT
    );
        '''


In [8]:
with engine.connect() as connection:
    connection.execute(query)
    print("Table created")

2025-03-19 14:15:34,376 INFO sqlalchemy.engine.Engine 
        CREATE TABLE IF NOT EXISTS reviews (
            id INT AUTO_INCREMENT PRIMARY KEY,
            date_published DATE,
            aircraft VARCHAR(255),
            type_of_traveller VARCHAR(100),
            seat_type VARCHAR(50),
            route VARCHAR(255),
            date_flown DATE NULL,
            seat_comfort_ranking INT NULL,
            cabin_staff_service_ranking INT NULL,
            food_beverages_ranking INT NULL,
            ground_services_ranking INT NULL,
            value_for_money INT,
            recommended VARCHAR(10),
            verification_status VARCHAR(50),
            review_text_clean TEXT,
            flight_type VARCHAR(50),
            departure VARCHAR(100),
            destination VARCHAR(100),
            layover VARCHAR(100) NULL,
            sentiment_score FLOAT,
            sentiment VARCHAR(10),
            keywords TEXT
    );
        
2025-03-19 14:15:34,377 INFO sqlalchemy.eng

### Note

This script checks whether the reviews table exists in the database. If it doesn't, it performs a first-time insert of all data from a CSV file. If the table already exists, it fetches existing records (using date_published and route as identifiers) and compares them with the new CSV data. Only new records are inserted into the database, preventing duplicates.

In [9]:
df = pd.read_csv("skytrax-data-with-sentiment.csv")

In [10]:
table_exists = engine.execute(
    "SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = 'SkytraxReviews' AND table_name = 'reviews'"
).fetchone()[0]


if table_exists == 0:
    df.to_sql("reviews", con=engine, if_exists="append", index=False)
    print("First-time insert completed")
else:
    existing_data = pd.read_sql("SELECT date_published, route FROM reviews", con=engine)

    df_new = df.merge(existing_data, on=["date_published", "route"], how="left", indicator=True)
    df_new = df_new[df_new["_merge"] == "left_only"].drop(columns=["_merge"])


    if not df_new.empty:
        df_new.to_sql("reviews", con=engine, if_exists="append", index=False)
        print("New data inserted")
    else:
        print("No new data found, nothing inserted")

2025-03-19 14:15:34,493 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = 'SkytraxReviews' AND table_name = 'reviews'
2025-03-19 14:15:34,494 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-19 14:15:34,497 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2025-03-19 14:15:34,497 INFO sqlalchemy.engine.Engine [generated in 0.00037s] {'table_schema': 'bank', 'table_name': 'SELECT date_published, route FROM reviews'}
2025-03-19 14:15:34,499 INFO sqlalchemy.engine.Engine SELECT date_published, route FROM reviews
2025-03-19 14:15:34,499 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-19 14:15:34,516 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2025-03-19 14:15:34,517 INFO sqlalchemy.engine.Engine [cached since 0.01978s ago] {'table_schema': '

2025-03-19 14:15:34,965 INFO sqlalchemy.engine.Engine COMMIT
New data inserted
